In [12]:
import configparser
import json
import requests
import pandas as pd
from huggingface_hub import InferenceClient

In [13]:
file_path = 'emails.pkl'
df = pd.read_pickle(file_path)
df.head(10)


,uid,to,from,subject,content,timestamp
0,16501,yashjoshi30052002@gmail.com,Medium Daily Digest <noreply@medium.com>,wake 5 every day bryan ye better h...,stories yash joshi yashjoshi30052002 ·become...,2025-04-13 01:40:00 UTC
1,16502,yashjoshi30052002@gmail.com,The Hindu <news@newsalertth.thehindu.com>,editors pick tamil nadu govt passes pending bi...,editors pick 13 april 2025 editors pick ne...,2025-04-13 04:39:53 UTC
2,16503,yashjoshi30052002@gmail.com,Prime Video <no-reply@primevideo.com>,recently added prime video watch,find next favourite prime video amazon pri...,2025-04-13 05:03:35 UTC
3,16504,yashjoshi30052002@gmail.com,YONO SBI <YONOSBI@communications.sbi.co.in>,single app simple savings solutions,text part,2025-04-13 11:19:38 UTC+05:30
4,16505,yashjoshi30052002@gmail.com,"""Coursera"" <Coursera@m.learn.coursera.org>",:alarm_clock: ends tomorrow get coursera plus ...,final chance unlock 10000 programs google ...,2025-04-13 06:12:57 UTC
5,16506,yashjoshi30052002@gmail.com,redBus <greetings@travel.e-redbus.in>,yash long weekend coming :backhand_index_poi...,redbus welcome plan bus journey upcoming...,2025-04-13 12:58:18 UTC+05:30
6,16507,yashjoshi30052002@gmail.com,Anthropic <no-reply-5l65bzxcBULVM9UnWStITA@mai...,secure link log claudeai 20250413 170302,doctype htmlhtml langund dirauto xmlns xmlnsvu...,2025-04-13 11:33:02 UTC
7,16508,yashjoshi30052002@gmail.com,huggingface <website@huggingface.co>,hugging face click link confirm email ad...,html head style html fontfamily serif fontsize...,2025-04-13 13:01:43 UTC
8,16509,yashjoshi30052002@gmail.com,huggingface <website@huggingface.co>,hugging face click link confirm email ad...,html head style html fontfamily serif fontsize...,2025-04-13 13:01:49 UTC
9,16510,yashjoshi30052002@gmail.com,Anthropic Team <team@email.anthropic.com>,you’re in—meet claude new ai assistant,welcome claude ai assistant designed hel...,2025-04-13 13:14:59 UTC


In [14]:

config = configparser.ConfigParser()
config.read("app.config")

type_labels = json.loads(config["analyse"]["type_labels"])
urgency_labels = json.loads(config["analyse"]["urgency_labels"])


API_URL_classification = "https://router.huggingface.co/hf-inference/models/facebook/bart-large-mnli"
headers = json.loads(config["analyse"]["headers"])

def type_classification(sequence ):
    payload = { "inputs": sequence,   "parameters": {"candidate_labels": type_labels }}
    response = requests.post(API_URL_classification, headers=headers, json=payload)
    result = response.json()
    return result['labels'][0], result['scores'][0]

def urgency_classification(sequence ):
    payload = { "inputs": sequence,   "parameters": {"candidate_labels": urgency_labels }}
    response = requests.post(API_URL_classification, headers=headers, json=payload)
    result = response.json()
    return result['labels'][0], result['scores'][0]


def sentiment(sequence ):
    client = InferenceClient(
    provider="hf-inference",
    api_key= config['analyse']['api_key']
)
    result = client.text_classification(
    text=sequence,
    model="cardiffnlp/twitter-roberta-base-sentiment-latest"
)
    return dict(result[0])['label'] , dict(result[0])['score']


In [ ]:
for i in range( 0 , df.shape[0]):

    sequence = sequence = df['content'][i]
    max_length = 600
    if len(sequence) > max_length:
        sequence = sequence[:max_length]

    class_type , class_confidence = type_classification(sequence)
    urgency_label , urgency_confidence = urgency_classification(sequence)
    sentiment_label , sentiment_score = sentiment(sequence)

    df.loc[i, 'type'] = class_type
    df.loc[i, 'type_confidence'] = class_confidence
    df.loc[i, 'sentiment'] = sentiment_label
    df.loc[i, 'sentiment_confidence'] = sentiment_score
    df.loc[i, 'urgency'] = urgency_label
    df.loc[i, 'urgency_confidence'] = urgency_confidence

    print(f'row {i} done')

df.head(20)


In [21]:
df.to_pickle("type.pkl")

In [20]:
df.head(10)

,uid,to,from,subject,content,timestamp,type,type_confidence,sentiment,sentiment_confidence,urgency,urgency_confidence
0,16501,yashjoshi30052002@gmail.com,Medium Daily Digest <noreply@medium.com>,wake 5 every day bryan ye better h...,stories yash joshi yashjoshi30052002 ·become...,2025-04-13 01:40:00 UTC,News,0.361855,neutral,0.875669,Minor Urgency,0.673190
1,16502,yashjoshi30052002@gmail.com,The Hindu <news@newsalertth.thehindu.com>,editors pick tamil nadu govt passes pending bi...,editors pick 13 april 2025 editors pick ne...,2025-04-13 04:39:53 UTC,Editorial,0.507610,neutral,0.904959,Major urgency,0.811579
2,16503,yashjoshi30052002@gmail.com,Prime Video <no-reply@primevideo.com>,recently added prime video watch,find next favourite prime video amazon pri...,2025-04-13 05:03:35 UTC,technology,0.511287,positive,0.857513,Major urgency,0.448127
3,16504,yashjoshi30052002@gmail.com,YONO SBI <YONOSBI@communications.sbi.co.in>,single app simple savings solutions,text part,2025-04-13 11:19:38 UTC+05:30,Blog,0.234043,neutral,0.854718,Minor Urgency,0.530852
4,16505,yashjoshi30052002@gmail.com,"""Coursera"" <Coursera@m.learn.coursera.org>",:alarm_clock: ends tomorrow get coursera plus ...,final chance unlock 10000 programs google ...,2025-04-13 06:12:57 UTC,technology,0.398501,neutral,0.735766,Minor Urgency,0.621216
5,16506,yashjoshi30052002@gmail.com,redBus <greetings@travel.e-redbus.in>,yash long weekend coming :backhand_index_poi...,redbus welcome plan bus journey upcoming...,2025-04-13 12:58:18 UTC+05:30,technology,0.553165,neutral,0.726214,Minor Urgency,0.417393
6,16507,yashjoshi30052002@gmail.com,Anthropic <no-reply-5l65bzxcBULVM9UnWStITA@mai...,secure link log claudeai 20250413 170302,doctype htmlhtml langund dirauto xmlns xmlnsvu...,2025-04-13 11:33:02 UTC,technology,0.363067,neutral,0.876292,Minor Urgency,0.535548
7,16508,yashjoshi30052002@gmail.com,huggingface <website@huggingface.co>,hugging face click link confirm email ad...,html head style html fontfamily serif fontsize...,2025-04-13 13:01:43 UTC,News,0.417665,neutral,0.854676,Minor Urgency,0.564411
8,16509,yashjoshi30052002@gmail.com,huggingface <website@huggingface.co>,hugging face click link confirm email ad...,html head style html fontfamily serif fontsize...,2025-04-13 13:01:49 UTC,News,0.417665,neutral,0.854676,Minor Urgency,0.564411
9,16510,yashjoshi30052002@gmail.com,Anthropic Team <team@email.anthropic.com>,you’re in—meet claude new ai assistant,welcome claude ai assistant designed hel...,2025-04-13 13:14:59 UTC,technology,0.343707,positive,0.870992,Minor Urgency,0.442512
